# Arquivo para treinamento do Yolo para reconhecer raças de cachorros

dataset: https://www.kaggle.com/jessicali9530/stanford-dogs-dataset

In [3]:
!pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 11.0/11.0 MB 85.8 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 12.6/12.6 MB 98.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ---------- ----------------------------- 12.1/44.5 MB 58.1 MB/s eta 0:00:01
   ----------------- ---------------------- 19.9/44.5 MB 48.4 MB/s eta 0:00:01
   ----------------------- ---------------- 26.5/44.5 MB 43.0 MB/s eta 0:00:01
   ---------------------------- ----------- 32.0/44.5 MB 39.0 MB/s eta 0:00:01
   --------------------------------- ------ 37.7/44.5 MB 36.9 MB/s eta 0:00:01
   ---------------------------------------  43.5/44.5 MB 35.5 MB/s eta 0:00:01
   ----------------------------------------


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Divisão do dataset

In [12]:
# O dataset inicialmente é composto 
import os
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET

In [14]:
dataset_dir = 'datasetReduced'
images_dir = os.path.join(dataset_dir, 'images')
print(images_dir)
annotations_dir = os.path.join(dataset_dir, 'labels')
print(annotations_dir)
# Racas n02085620-Chihuahua, n02086240-Shih-Tzu, n02094433-Yorkshire_terrier, n02099601-golden_retriever, n02106166-Border_collie, n02106550-Rottweiler, n02107142-Doberman, n02108089-boxer, n02109047-Great_Dane, n02110958-pug
dog_classes = ['n02085620', 'n02086240', 'n02094433', 'n02099601', 'n02106166', 'n02106550', 'n02107142', 'n02108089', 'n02109047', 'n02110958']
dog_names = ['Chihuahua', 'Shih-Tzu', 'Yorkshire_terrier', 'golden_retriever', 'Border_collie', 'Rottweiler', 'Doberman', 'boxer', 'Great_Dane', 'pug']

datasetReduced\images
datasetReduced\labels


In [15]:
def list_files(directory, extension):
    files = []
    for root, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith(extension):
                files.append(os.path.relpath(os.path.join(root, filename), directory))
    return files

def convert_xml_to_yolo(xml_file, classes):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    yolo_annotations = []
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_id = classes.index(class_name)

        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        x_center = (xmin + xmax) / 2.0 / width
        y_center = (ymin + ymax) / 2.0 / height
        bbox_width = (xmax - xmin) / float(width)
        bbox_height = (ymax - ymin) / float(height)

        yolo_annotations.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")

    return yolo_annotations


In [16]:
for root, _, files in os.walk(annotations_dir):
    for file in files:
        xml_path = os.path.join(root, file)
        
        # Verificar se o arquivo é XML
        try:
            tree = ET.parse(xml_path)
            print(f"Arquivo XML encontrado: {xml_path}")
            yolo_annotations = convert_xml_to_yolo(xml_path, dog_names)

            # Salvar anotações YOLO em arquivo .txt correspondente
            yolo_path = os.path.splitext(xml_path)[0] + '.txt'
            print(f"Salvando anotações YOLO em: {yolo_path}")
            with open(yolo_path, 'w') as f:
                f.write('\n'.join(yolo_annotations))
        except ET.ParseError:
            print(f"Arquivo não é um XML válido: {xml_path}")

print("Conversão das anotações concluída.")

Arquivo XML encontrado: datasetReduced\labels\n02085620-Chihuahua\n02085620_10074
Salvando anotações YOLO em: datasetReduced\labels\n02085620-Chihuahua\n02085620_10074.txt
Arquivo XML encontrado: datasetReduced\labels\n02085620-Chihuahua\n02085620_10131
Salvando anotações YOLO em: datasetReduced\labels\n02085620-Chihuahua\n02085620_10131.txt
Arquivo XML encontrado: datasetReduced\labels\n02085620-Chihuahua\n02085620_10621
Salvando anotações YOLO em: datasetReduced\labels\n02085620-Chihuahua\n02085620_10621.txt
Arquivo XML encontrado: datasetReduced\labels\n02085620-Chihuahua\n02085620_1073
Salvando anotações YOLO em: datasetReduced\labels\n02085620-Chihuahua\n02085620_1073.txt
Arquivo XML encontrado: datasetReduced\labels\n02085620-Chihuahua\n02085620_10976
Salvando anotações YOLO em: datasetReduced\labels\n02085620-Chihuahua\n02085620_10976.txt
Arquivo XML encontrado: datasetReduced\labels\n02085620-Chihuahua\n02085620_11140
Salvando anotações YOLO em: datasetReduced\labels\n02085620-

In [17]:
# Listar todas as imagens e anotações
images = list_files(images_dir, '.jpg')
print(images)
annotations = list_files(annotations_dir, '.txt')
print(annotations)

# Dividir em treino e validação (80% treino, 20% validação)
train_images, val_images, train_annotations, val_annotations = train_test_split(
    images, annotations, test_size=0.2, random_state=42
)

# Criar pastas de treino e validação
train_images_dir = os.path.join(images_dir, 'train')
val_images_dir = os.path.join(images_dir, 'val')
train_annotations_dir = os.path.join(annotations_dir, 'train')
val_annotations_dir = os.path.join(annotations_dir, 'val')

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(train_annotations_dir, exist_ok=True)
os.makedirs(val_annotations_dir, exist_ok=True)

def move_files(files, src_dir, dst_dir):
    for file in files:
        src_path = os.path.join(src_dir, file)
        dst_path = os.path.join(dst_dir, file)
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        shutil.move(src_path, dst_path)

# Mover arquivos para as pastas correspondentes
move_files(train_images, images_dir, train_images_dir)
move_files(val_images, images_dir, val_images_dir)
move_files(train_annotations, annotations_dir, train_annotations_dir)
move_files(val_annotations, annotations_dir, val_annotations_dir)

print("Divisão do dataset concluída.")

['n02085620-Chihuahua\\n02085620_10074.jpg', 'n02085620-Chihuahua\\n02085620_10131.jpg', 'n02085620-Chihuahua\\n02085620_10621.jpg', 'n02085620-Chihuahua\\n02085620_1073.jpg', 'n02085620-Chihuahua\\n02085620_10976.jpg', 'n02085620-Chihuahua\\n02085620_11140.jpg', 'n02085620-Chihuahua\\n02085620_11238.jpg', 'n02085620-Chihuahua\\n02085620_11258.jpg', 'n02085620-Chihuahua\\n02085620_11337.jpg', 'n02085620-Chihuahua\\n02085620_11477.jpg', 'n02085620-Chihuahua\\n02085620_1152.jpg', 'n02085620-Chihuahua\\n02085620_11696.jpg', 'n02085620-Chihuahua\\n02085620_11818.jpg', 'n02085620-Chihuahua\\n02085620_11948.jpg', 'n02085620-Chihuahua\\n02085620_1205.jpg', 'n02085620-Chihuahua\\n02085620_12101.jpg', 'n02085620-Chihuahua\\n02085620_12334.jpg', 'n02085620-Chihuahua\\n02085620_1235.jpg', 'n02085620-Chihuahua\\n02085620_1271.jpg', 'n02085620-Chihuahua\\n02085620_12718.jpg', 'n02085620-Chihuahua\\n02085620_1298.jpg', 'n02085620-Chihuahua\\n02085620_13151.jpg', 'n02085620-Chihuahua\\n02085620_1321.